# Ability of Artificial Intelligence to Identify Self-Reported Race in Chest X-Ray Using Pixel Intensity Counts
## Tensorflow Feed Forward Network and Decision Tree

In [ ]:
#!pip install keras-tuner --upgrade

In [ ]:
## Server specific Keras requirement

import os
import GPUtil
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
DEVICE_ID_LIST = GPUtil.getAvailable(order = 'memory', limit = 2,
maxMemory = 0.5 )
LIST_LENGTH = len(DEVICE_ID_LIST)
if LIST_LENGTH == 0 :
    raise ValueError("There are no available GPUs with the required limits listed in GPUtil.getAvailable()")
AVAIL_DEVICES = str(DEVICE_ID_LIST[0]) # grab first element from list
#intentionally starting with 1 because we've added 0
COUNT = 1
while COUNT < LIST_LENGTH:
    AVAIL_DEVICES += "," + str(DEVICE_ID_LIST[COUNT])
    COUNT += 1
os.environ["CUDA_VISIBLE_DEVICES"] = AVAIL_DEVICES
print('Device IDs (unmasked): ' + AVAIL_DEVICES)

In [ ]:
#imports
import numpy as np
import pandas as pd
import tensorflow as tf
import keras_tuner as kt
from tensorflow.keras.layers.experimental import preprocessing
from sklearn.utils.class_weight import compute_class_weight
from sklearn.metrics import accuracy_score, confusion_matrix, roc_curve, auc
from sklearn.datasets import fetch_lfw_people
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, Activation, Conv1D, Conv2D, MaxPool1D, MaxPooling2D, AveragePooling1D, Flatten, Dropout, BatchNormalization, LSTM, GRU, Softmax, RNN, SimpleRNN, LSTM, GRU
from tensorflow.keras.regularizers import l1, l2, l1_l2
from keras.models import Sequential
from tensorflow.keras.optimizers import SGD, RMSprop, Adam, Adamax, Ftrl
import matplotlib.pyplot as plt

np.random.seed(42)
tf.random.set_seed(42)

In [ ]:
# read from a csv
freq_df = pd.read_csv('frequencies.csv', index_col=0, header=0)

#get all race labels
races = freq_df['race'].unique()

#convert to percents
#first make a copy of data
#get just the numeric columns - removing 0 to remove all pure non-image black space often used in border to rotate the images
freq_percent_df = freq_df.copy(deep=True).iloc[:, 1 : 257]

#get just the numeric columns
freq_percent_df_num = freq_percent_df.iloc[:, 0 : 255]

#conversion to percents
freq_percent_df_num = freq_percent_df_num.div(freq_df.sum(axis=1, numeric_only=True), axis=0)
freq_percent_df[freq_percent_df_num.columns] = freq_percent_df_num


#drop classes, for Black or white analysis uncomment line 21 and comment line 22
#freq_percent_df2 = freq_percent_df.copy(deep=True)[freq_percent_df['race'].str.contains("asian|hispanic")==False]
freq_percent_df2 = freq_percent_df.copy(deep=True)


#get the x and y values
x = freq_percent_df2.iloc[:, 0 : 255].values
y = freq_percent_df2['race']

#Black or other setup
y.replace({'asian': 0, 'black': 1, 'hispanic': 0, 'white':0}, inplace=True)

x_train, x_test, y_train, y_test = train_test_split(x, y, stratify=y, test_size=0.1)

# Converting the type to 'float'
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')

#### Using keras_tuner to Derive Best FFN
https://keras.io/guides/keras_tuner/getting_started/

In [ ]:
outputs = 1 #number of races

class MyHyperModel(kt.HyperModel):
    def build(self, hp):
        model = Sequential()
        model.add(Dense(units=hp.Int("units_0", min_value=512, max_value=1024, step=2), input_shape=(255,)))
        for i in range(hp.Int("num_layers", 0, 2)):
            model.add(Dense(units=hp.Int("units_1", min_value=512, max_value=2048, step=3)))
            model.add(Activation(hp.Choice("activation_1", ["relu", "tanh", "sigmoid"])))
        for i in range(hp.Int("num_layers2", 0, 2)):
            model.add(Dense(units=hp.Int("units_2", min_value=512, max_value=2048, step=3)))
            model.add(Activation(hp.Choice("activation_2", ["relu", "tanh", "sigmoid"])))
        if hp.Boolean("dropout"):
            model.add(Dropout(hp.Float("dropout_val", min_value=0.01, max_value=0.25, step=4)))
        if hp.Boolean("regularizer"):
            model.add(Dense(hp.Int("units_2", min_value=512, max_value=2048, step=3), kernel_regularizer=l2(hp.Float("regularizer_int", min_value=0.0001, max_value=0.1, step=4))))
        if hp.Boolean("batch_norm"):
            model.add(BatchNormalization())
        if hp.Boolean("next_to_last_activation"):
            model.add(Activation(hp.Choice("activation_nlt", ["relu", "tanh","sigmoid"])))
        if hp.Boolean("next_to_last_dense"):
            model.add(Dense(units=hp.Int("units_ntl", min_value=256, max_value=2048, step=4)))
        model.add(Dense(outputs, activation='sigmoid'))
        
        optimizer = Adam(
            learning_rate=0.0001,
            beta_1=0.9,
            beta_2=0.999,
            epsilon=1e-07,
            amsgrad=True
            )
        
        model.compile(optimizer=optimizer, 
                      loss='binary_crossentropy', 
                      metrics=['accuracy', 'AUC'])
        return model
    
    def fit(self, hp, model, *args, **kwargs):
        return model.fit(
            *args,
            shuffle=hp.Boolean("shuffle"),
            **kwargs,
        )

In [ ]:
#history_ffn = model_ffn.fit(x=x_train,y=y_train, epochs=160,validation_split=0.05, batch_size=1024, shuffle=True)

hp = kt.HyperParameters()
hypermodel = MyHyperModel()
model = hypermodel.build(hp)
hypermodel.fit(hp, model, x=x_train,y=y_train, epochs=120,validation_split=0.2, batch_size=2048)

In [ ]:
tuner = kt.RandomSearch(
    MyHyperModel(),
    objective=kt.Objective("val_auc", direction="max"),
    max_trials=100,
    overwrite=False,
    directory='models_bva',
    project_name='H518_final_bva'
)

#tuner.search(x=x_train, y=y_train, validation_split=0.2, epochs=60, batch_size=4096)

In [ ]:
# Get the top 2 models.
models = tuner.get_best_models(num_models=2)
best_model = models[0]
# Build the model.
best_model.build()
best_model.summary()

In [ ]:
tuner.results_summary(1)

In [ ]:
best_model.evaluate(x=x_test, y=y_test, return_dict=True)

### Feed Forward Network

In [ ]:
outputs = 1 #number of races
hidden_size = 1024
hidden_size2 = 2048
hidden_size3 = 4096

model_ffn = Sequential()
model_ffn.add(Dense(hidden_size, input_shape=(255,)))
model_ffn.add(Dense(hidden_size))
model_ffn.add(Activation('relu'))
model_ffn.add(Dense(hidden_size))
model_ffn.add(Activation('relu'))
model_ffn.add(Dense(hidden_size))
model_ffn.add(Activation('tanh'))
model_ffn.add(Dropout(0.01))
model_ffn.add(Dense(hidden_size, kernel_regularizer=l2(0.0001)))
model_ffn.add(Dense(outputs, activation='sigmoid'))


optimizer = Adam(
    learning_rate=0.0001,
    beta_1=0.9,
    beta_2=0.999,
    epsilon=1e-07,
    #amsgrad=True
    )

#binary_crossentropy for binary
model_ffn.compile(optimizer=optimizer, 
              loss='binary_crossentropy', 
              metrics=['accuracy', 'AUC'])

model_ffn.summary()

In [ ]:
history_ffn = model_ffn.fit(x=x_train,y=y_train, epochs=100,validation_split=0.2, batch_size=4096, shuffle=True)

model_ffn.evaluate(x=x_test, y=y_test, return_dict=True)

preds0 = model_ffn.predict(x_test)

In [ ]:
fpr, tpr, thresholds = roc_curve(y_test, preds)
auc_calc = auc(fpr, tpr)
auc_calc

In [ ]:
plt.figure(1)
plt.plot([0, 1], [0, 1], 'k--')
plt.plot(fpr, tpr, label='Binary (area = {:.3f})'.format(auc_calc))
plt.xlabel('False positive rate')
plt.ylabel('True positive rate')
plt.title('ROC curve')
plt.legend(loc='best')
plt.show()

### Decision Trees

In [ ]:
label = "race"
classes = freq_percent_df2[label].unique().tolist()
print(f"Label classes: {classes}")

freq_percent_df2[label] = freq_percent_df2[label].map(classes.index)

def split_dataset(dataset, test_ratio=0.10):
  """Splits a panda dataframe in two."""
  test_indices = np.random.rand(len(dataset)) < test_ratio
  return dataset[~test_indices], dataset[test_indices]


train_ds_pd, test_ds_pd = split_dataset(freq_percent_df2)
print("{} examples in training, {} examples for testing.".format(
    len(train_ds_pd), len(test_ds_pd)))

train_ds = tfdf.keras.pd_dataframe_to_tf_dataset(train_ds_pd, label=label)
test_ds = tfdf.keras.pd_dataframe_to_tf_dataset(test_ds_pd, label=label)

In [ ]:
# Testing 3 models in one run
#RandomForestModel
model_1 = tfdf.keras.RandomForestModel(hyperparameter_template="benchmark_rank1")
#GradientBoostedTreesModel
model_2 = tfdf.keras.GradientBoostedTreesModel(hyperparameter_template="benchmark_rank1")
#CartModel
model_3 = tfdf.keras.CartModel()

# Optionally, add evaluation metrics.
model_1.compile(
    metrics=["AUC", "accuracy"])
model_2.compile(
    metrics=["AUC", "accuracy"])
model_3.compile(
    metrics=["AUC", "accuracy"])

# Train the model.
# "sys_pipes" is optional. It enables the display of the training logs.
with sys_pipes():
    model_1.fit(x=train_ds)
    model_2.fit(x=train_ds)
    model_3.fit(x=train_ds)

In [ ]:
evaluation1 = model_1.evaluate(test_ds, return_dict=True)
print()

for name, value in evaluation1.items():
  print(f"Random Forest: {name}: {value:.4f}")

evaluation2 = model_2.evaluate(test_ds, return_dict=True)
print()

for name, value in evaluation2.items():
  print(f"GradientBoostedTreesModel: {name}: {value:.4f}")

evaluation3 = model_3.evaluate(test_ds, return_dict=True)
print()

for name, value in evaluation3.items():
  print(f"CartModel: {name}: {value:.4f}")